In [30]:
from statsmodels.regression.rolling import RollingOLS
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import yfinance as yf   
import datetime as dt

#%pip install pandas_ta --upgrade --quiet
import pandas_ta
import warnings
warnings.filterwarnings("ignore")

In [31]:
# Get the S&P 500 tickers from Wikipedia
# This will fetch the current list of S&P 500 companies
sp500Tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500Tickers = sp500Tickers[0]
sp500Tickers['Symbol'] = sp500Tickers['Symbol'].str.replace('.', '-').unique()
sp500Tickers = sp500Tickers['Symbol'].to_list()

# Set the start and end dates for the data
end_date = dt.datetime(2024, 12, 31)
start_date = end_date - dt.timedelta(days=365*8)  # 5 years of data

In [ ]:
# download the data
data = yf.download(sp500Tickers, start=start_date, end=end_date)
data

[********************* 44%                       ]  220 of 502 completed

KeyboardInterrupt: 

[*********************100%***********************]  501 of 502 completed

In [71]:
df = data.copy()
df = df.stack()
# make Date', 'Ticker' the indexes
df.index.names = ['Date', 'Ticker']
df.columns = df.columns.str.lower()
df

Price                   close        high         low        open       volume
Date       Ticker                                                             
2017-01-03 A        43.559395   43.803004   42.856673   43.034694    1739600.0
           AAPL     26.827248   26.868822   26.506198   26.746408  115127600.0
           ABBV     43.450222   43.881869   43.123004   43.805286    9328200.0
           ABT      33.456711   33.482416   32.848408   33.096871    9677300.0
           ACGL     27.224224   27.712353   27.106947   27.522172     942900.0
...                       ...         ...         ...         ...          ...
2024-12-30 XYL     115.551468  116.068214  114.438490  115.730340     586800.0
           YUM     132.243332  133.124821  131.728300  132.847500    1144600.0
           ZBH     104.902634  105.848148  104.156173  105.549568    1532000.0
           ZBRA    383.850006  386.959991  378.149994  385.059998     211300.0
           ZTS     161.207016  163.005489  160.422037  162.846504    1531400.0

[986646 rows x 5 columns]

### Technical indicators:

- Garman-Klaus:
$$ GKV = \frac{(\ln(High)-\ln(Low))^2}{2}- (2\ln(2)-1)(\ln(AQdj Close)-\ln(Open))^2$$

In [72]:
# Geman-Klass volatility calculation
df['garman_klass_vol'] = 0.5*(np.log(df['high'])-np.log(df['low']))**2 -\
      (2*np.log(2)-1)*(np.log(df['close']-np.log(df['open'])))**2

# RSI calculation using pandas_ta
df['rsi'] = df.groupby(level=1)['close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

# Bolinger Bands calculation using pandas_ta
df['bb_lower'] = \
    df.groupby(level=1)['close'].transform(lambda x: pandas_ta.bbands(close=x, length=20, std=2).iloc[:, 0].T.values)
df['bb_middle']= \
    df.groupby(level=1)['close'].transform(lambda x: pandas_ta.bbands(close=x, length=20, std=2).iloc[:, 1].T.values)
df['bb_upper'] = \
    df.groupby(level=1)['close'].transform(lambda x: pandas_ta.bbands(close=x, length=20, std=2).iloc[:, 2].T.values)
# Normalised ATR calculation using pandas_ta
def compute_ATR(stock_df):
    atr = pandas_ta.atr(high=stock_df['high'], 
                        low=stock_df['low'], 
                        close=stock_df['close'], 
                        length=20)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_ATR)

#Compute Normalised MACD using pandas_ta
def calc_MACD(stock_df):
    macd = pandas_ta.macd(close=stock_df, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['close'].apply(calc_MACD)

# Compute dollar-volume
df['dollar_vol'] = df['close'] * df['volume']/1e6 # in millions

df

Price                   close        high         low        open  \
Date       Ticker                                                   
2017-01-03 A        43.559395   43.803004   42.856673   43.034694   
           AAPL     26.827248   26.868822   26.506198   26.746408   
           ABBV     43.450222   43.881869   43.123004   43.805286   
           ABT      33.456711   33.482416   32.848408   33.096871   
           ACGL     27.224224   27.712353   27.106947   27.522172   
...                       ...         ...         ...         ...   
2024-12-30 XYL     115.551468  116.068214  114.438490  115.730340   
           YUM     132.243332  133.124821  131.728300  132.847500   
           ZBH     104.902634  105.848148  104.156173  105.549568   
           ZBRA    383.850006  386.959991  378.149994  385.059998   
           ZTS     161.207016  163.005489  160.422037  162.846504   

Price                   volume  garman_klass_vol        rsi    bb_lower  \
Date       Ticker                                                         
2017-01-03 A         1739600.0         -5.241927        NaN         NaN   
           AAPL    115127600.0         -3.854206        NaN         NaN   
           ABBV      9328200.0         -5.232927        NaN         NaN   
           ABT       9677300.0         -4.464781        NaN         NaN   
           ACGL       942900.0         -3.892041        NaN         NaN   
...                        ...               ...        ...         ...   
2024-12-30 XYL        586800.0         -8.561229  35.375387  112.122510   
           YUM       1144600.0         -9.075210  46.389793  129.571023   
           ZBH       1532000.0         -8.200897  43.454500  103.925319   
           ZBRA       211300.0        -13.604915  45.199475  381.008820   
           ZTS       1531400.0         -9.853638  32.695526  158.582717   

Price               bb_middle    bb_upper       atr      macd   dollar_vol  
Date       Ticker                                                           
2017-01-03 A              NaN         NaN       NaN       NaN    75.775923  
           AAPL           NaN         NaN       NaN       NaN  3088.556633  
           ABBV           NaN         NaN       NaN       NaN   405.312361  
           ABT            NaN         NaN       NaN       NaN   323.770628  
           ACGL           NaN         NaN       NaN       NaN    25.669721  
...                       ...         ...       ...       ...          ...  
2024-12-30 XYL     121.125830  130.129150  0.668220 -1.585003    67.805601  
           YUM     134.685751  139.800480  0.789449 -0.514963   151.365718  
           ZBH     107.000074  110.074829 -0.617094 -0.307769   160.710835  
           ZBRA    399.536504  418.064187  0.047316 -0.332876    81.107506  
           ZTS     170.540209  182.497701  0.484708 -1.758412   246.872424  

[986646 rows x 13 columns]

### Aggregate Indicators and Filter top 150 most liquid stocks (monthly)

- Convert daily data to monthly data

In [73]:
#Create list of columns we will take last monthly value
monthly_cols = [col for col in df.columns.unique() if col not in ['open', 'high', 'low' 
                                                                  , 'volume', 'dollar_vol']]
temp2 = df.unstack()[monthly_cols].resample('M').last().stack()
# Resample dollar-vol monthly
temp1 = df.unstack('Ticker')['dollar_vol'].resample('M').mean().stack().to_frame('dollar_vol.M')

df2 = pd.concat([temp1, temp2],
          axis=1)
df2

dollar_vol.M       close  garman_klass_vol        rsi  \
Date       Ticker                                                          
2017-01-31 A         101.729834   45.883072         -5.401556        NaN   
           AAPL     3113.181593   28.028296         -3.972143        NaN   
           ABBV      338.538888   42.974422         -5.201184        NaN   
           ABT       368.277086   36.018269         -4.677740        NaN   
           ACGL       29.688113   28.003962         -3.968958        NaN   
...                         ...         ...               ...        ...   
2024-12-31 XYL       165.083143  115.551468         -8.561229  35.375387   
           YUM       206.888270  132.243332         -9.075210  46.389793   
           ZBH       164.685227  104.902634         -8.200897  43.454500   
           ZBRA      121.040691  383.850006        -13.604915  45.199475   
           ZTS       414.343721  161.207016         -9.853638  32.695526   

                     bb_lower   bb_middle    bb_upper       atr      macd  
Date       Ticker                                                          
2017-01-31 A        43.585910   44.939543   46.293175       NaN       NaN  
           AAPL     26.805020   27.617166   28.429313       NaN       NaN  
           ABBV     41.996999   43.386470   44.775942       NaN       NaN  
           ABT      33.751237   34.937111   36.122985       NaN       NaN  
           ACGL     26.968826   27.667502   28.366179       NaN       NaN  
...                       ...         ...         ...       ...       ...  
2024-12-31 XYL     112.122510  121.125830  130.129150  0.668220 -1.585003  
           YUM     129.571023  134.685751  139.800480  0.789449 -0.514963  
           ZBH     103.925319  107.000074  110.074829 -0.617094 -0.307769  
           ZBRA    381.008820  399.536504  418.064187  0.047316 -0.332876  
           ZTS     158.582717  170.540209  182.497701  0.484708 -1.758412  

[47108 rows x 9 columns]

#### Step 3: 5-year average rolling dollar-vol average

In [74]:
# 5-year average rolling dollar-vol average
df2['dollar_vol.M'] = df2['dollar_vol.M'].unstack('Ticker').rolling(5*12).mean().stack()

# Filter top 150 most liquid stocks
df2['liquidity_rank'] = df2.groupby(level=0)['dollar_vol.M'].rank(ascending=False)

#df2 = df2[df2['liquidity_rank'] < 150].drop(['dollar_vol.M', 'liquidity_rank'], axis=1).head()
df2



dollar_vol.M       close  garman_klass_vol        rsi  \
Date       Ticker                                                          
2017-01-31 A                NaN   45.883072         -5.401556        NaN   
           AAPL             NaN   28.028296         -3.972143        NaN   
           ABBV             NaN   42.974422         -5.201184        NaN   
           ABT              NaN   36.018269         -4.677740        NaN   
           ACGL             NaN   28.003962         -3.968958        NaN   
...                         ...         ...               ...        ...   
2024-12-31 XYL       121.918521  115.551468         -8.561229  35.375387   
           YUM       200.068570  132.243332         -9.075210  46.389793   
           ZBH       177.622847  104.902634         -8.200897  43.454500   
           ZBRA      124.836068  383.850006        -13.604915  45.199475   
           ZTS       343.506090  161.207016         -9.853638  32.695526   

                     bb_lower   bb_middle    bb_upper       atr      macd  \
Date       Ticker                                                           
2017-01-31 A        43.585910   44.939543   46.293175       NaN       NaN   
           AAPL     26.805020   27.617166   28.429313       NaN       NaN   
           ABBV     41.996999   43.386470   44.775942       NaN       NaN   
           ABT      33.751237   34.937111   36.122985       NaN       NaN   
           ACGL     26.968826   27.667502   28.366179       NaN       NaN   
...                       ...         ...         ...       ...       ...   
2024-12-31 XYL     112.122510  121.125830  130.129150  0.668220 -1.585003   
           YUM     129.571023  134.685751  139.800480  0.789449 -0.514963   
           ZBH     103.925319  107.000074  110.074829 -0.617094 -0.307769   
           ZBRA    381.008820  399.536504  418.064187  0.047316 -0.332876   
           ZTS     158.582717  170.540209  182.497701  0.484708 -1.758412   

                   liquidity_rank  
Date       Ticker                  
2017-01-31 A                  NaN  
           AAPL               NaN  
           ABBV               NaN  
           ABT                NaN  
           ACGL               NaN  
...                           ...  
2024-12-31 XYL              383.0  
           YUM              260.0  
           ZBH              286.0  
           ZBRA             375.0  
           ZTS              146.0  

[47108 rows x 10 columns]

In [75]:
df2.shape

(47108, 10)

#### Step 4: Calculate monthly returns for different time horizons

- Captures time series dynamics like Momentum patterns

In [82]:
# subset one ticker: AAPL
def calculate_returns(temp1):

    outlier_cut_off = 0.005

    lags = [1, 2, 3, 6, 9, 12] #Months
    for lag in lags:
        temp1[f'returns.{lag}M'] = (temp1['close']
                                    .pct_change(lag)
                                    .pipe(lambda x: x.clip(lower=x.quantile(outlier_cut_off),
                                                        upper=x.quantile(1-outlier_cut_off)))
                                    .add(1)
                                    .pow(1/lag)
                                    .sub(1)
        )
    temp1 = temp1.dropna()
    return temp1


df2 = df2.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()
df2.head(10)


,,dollar_vol.M,close,garman_klass_vol,rsi,bb_lower,bb_middle,bb_upper,atr,macd,liquidity_rank,returns.1M,returns.2M,returns.3M,returns.6M,returns.9M,returns.12M
Date,Ticker,,,,,,,,,,,,,,,,
2022-12-31,A,185.209115,146.924484,-9.485599,52.335580,142.719678,148.639553,154.559429,1.139258,0.056684,230.0,-0.032952,0.040812,0.072935,0.040134,0.014499,-0.004723
2023-01-31,A,186.313181,149.310242,-9.549060,50.155070,145.060098,151.209024,157.357950,1.347065,0.318483,231.0,0.016238,-0.008662,0.032555,0.021740,0.027939,0.007880
2023-02-28,A,185.757664,139.384369,-9.279307,38.286626,136.802937,146.956905,157.110873,0.977256,-1.764621,231.0,-0.066478,-0.025998,-0.028321,0.017629,0.012524,0.007686
2023-03-31,A,186.938471,135.820465,-9.178874,47.744395,127.216533,133.473030,139.729528,0.998830,-1.562482,232.0,-0.025569,-0.046243,-0.025855,0.022347,0.017656,0.004257
2023-04-30,A,187.483130,133.180084,-9.102755,46.269581,130.117347,135.141214,140.165081,0.840534,-0.725704,233.0,-0.019440,-0.022509,-0.037391,-0.003031,0.001636,0.011202
2023-05-31,A,188.418287,113.748375,-8.501750,27.663967,114.469602,124.745094,135.020586,0.944601,-2.379539,233.0,-0.142456,-0.084855,-0.064631,-0.047094,-0.010873,-0.007574
2023-06-30,A,190.057024,118.478096,-8.655869,46.545855,113.096482,116.323841,119.551200,0.322707,-0.976063,233.0,0.041581,-0.056810,-0.044514,-0.035230,-0.000444,0.001744
2023-07-31,A,192.102633,119.975700,-8.702160,48.433700,112.720085,120.262903,127.805720,0.630346,0.662354,233.0,0.012640,0.027009,-0.034206,-0.035800,-0.013533,-0.007447
2023-08-31,A,193.232283,119.286018,-8.680616,47.113446,114.104546,121.066875,128.029205,0.176147,-0.590716,234.0,-0.005749,0.003404,0.015971,-0.025618,-0.026520,-0.004229


#### Step 5: Download Fama-French Factors and Calc Rolling Factor beta

- Calculate exposure of assets to common risk factors
- Five Fama-French Factors:
    - Market risk
    - Size
    - Value
    - Operating Profitability
    - Investment
- Used to assess risk/return profiles of portfolios
- Access using `pandas-datareader`

In [ ]:
#### 

In [ ]:
help(pandas_ta.rsi)

Help on function rsi in module pandas_ta.momentum.rsi:

rsi(close, length=None, scalar=None, talib=None, drift=None, offset=None, **kwargs)
    Relative Strength Index (RSI)

    The Relative Strength Index is popular momentum oscillator used to measure the
    velocity as well as the magnitude of directional price movements.

    Sources:
        https://www.tradingview.com/wiki/Relative_Strength_Index_(RSI)

    Calculation:
        Default Inputs:
            length=14, scalar=100, drift=1
        ABS = Absolute Value
        RMA = Rolling Moving Average

        diff = close.diff(drift)
        positive = diff if diff > 0 else 0
        negative = diff if diff < 0 else 0

        pos_avg = RMA(positive, length)
        neg_avg = ABS(RMA(negative, length))

        RSI = scalar * pos_avg / (pos_avg + neg_avg)

    Args:
        close (pd.Series): Series of 'close's
        length (int): It's period. Default: 14
        scalar (float): How much to magnify. Default: 100
        talib 